In [110]:
import matplotlib.pyplot as plt
import numpy  as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, cross_val_predict,cross_validate, GridSearchCV

In [3]:
housing = pd.read_csv('../data/Ames_Housing_Price_Data.csv', index_col = 0)
real_estate = pd.read_csv('../data/Ames_Real_Estate_Data.csv', index_col = 0)

D:\software\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
correlation = housing.corr()['SalePrice']
# corr_matrix['SalePrice']
correlation.sort_values()

PID             -0.226276
EnclosedPorch   -0.123235
KitchenAbvGr    -0.113977
OverallCond     -0.101091
MSSubClass      -0.084587
BsmtHalfBath    -0.033802
LowQualFinSF    -0.030835
MiscVal         -0.018877
YrSold          -0.001376
MoSold           0.012591
BsmtFinSF2       0.022520
PoolArea         0.030844
3SsnPorch        0.031260
ScreenPorch      0.121463
BedroomAbvGr     0.151806
BsmtUnfSF        0.167519
2ndFlrSF         0.263144
LotArea          0.274443
HalfBath         0.285875
BsmtFullBath     0.291080
OpenPorchSF      0.306540
WoodDeckSF       0.333232
LotFrontage      0.358577
BsmtFinSF1       0.460792
Fireplaces       0.488184
TotRmsAbvGrd     0.490219
MasVnrArea       0.500964
YearRemodAdd     0.514309
GarageYrBlt      0.520763
FullBath         0.534982
YearBuilt        0.544370
GarageArea       0.634992
GarageCars       0.638801
1stFlrSF         0.642613
TotalBsmtSF      0.652222
GrLivArea        0.719981
OverallQual      0.790501
SalePrice        1.000000
Name: SalePr

Danny

In [75]:
housing3 = pd.concat([housing[housing.columns[53:]],housing['SalePrice']],axis=1)
housing3.columns

Index(['KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional',
       'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscFeature', 'MiscVal',
       'MoSold', 'YrSold', 'SaleType', 'SaleCondition', 'SalePrice'],
      dtype='object')

In [6]:
housing_features = housing[housing.columns[53:]]
housing_features = housing_features.rename(columns= {"3SsnPorch":"X3SsnPorch"})
housing_featuresClass = housing_features[['KitchenQual','Functional','FireplaceQu','GarageType','GarageFinish','GarageQual',\
                                         'GarageCond','PavedDrive','PoolQC','Fence','MiscFeature','SaleType','SaleCondition']]
housing_featuresQuan = housing_features[['KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageYrBlt','GarageCars','GarageArea',\
                                        'WoodDeckSF','OpenPorchSF','EnclosedPorch','X3SsnPorch','ScreenPorch','PoolArea',\
                                         'MiscVal','MoSold','YrSold']]

In [59]:
housing_features.columns

Index(['KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional',
       'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
       'X3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence',
       'MiscFeature', 'MiscVal', 'MoSold', 'YrSold', 'SaleType',
       'SaleCondition'],
      dtype='object')

In [7]:
for feature1 in housing_featuresClass.columns:
    print(housing3[feature1].value_counts(), housing_featuresClass[feature1].isnull().value_counts())
    print('*'*30)

TA    1358
Gd    1017
Ex     142
Fa      62
Po       1
Name: KitchenQual, dtype: int64 False    2580
Name: KitchenQual, dtype: int64
******************************
Typ     2399
Min2      66
Min1      57
Mod       32
Maj1      18
Maj2       7
Sal        1
Name: Functional, dtype: int64 False    2580
Name: Functional, dtype: int64
******************************
Gd    619
TA    572
Fa     70
Po     43
Ex     35
Name: FireplaceQu, dtype: int64 False    1339
True     1241
Name: FireplaceQu, dtype: int64
******************************
Attchd     1527
Detchd      716
BuiltIn     153
Basment      27
2Types       21
CarPort       9
Name: GarageType, dtype: int64 False    2453
True      127
Name: GarageType, dtype: int64
******************************
Unf    1112
RFn     718
Fin     621
Name: GarageFinish, dtype: int64 False    2451
True      129
Name: GarageFinish, dtype: int64
******************************
TA    2312
Fa     110
Gd      22
Po       4
Ex       3
Name: GarageQual, dtype: int64 F

In [8]:
for feature2 in housing_featuresQuan.columns:
    print(housing_featuresQuan[feature2].isnull().value_counts())
    print('*'*30)

False    2580
Name: KitchenAbvGr, dtype: int64
******************************
False    2580
Name: TotRmsAbvGrd, dtype: int64
******************************
False    2580
Name: Fireplaces, dtype: int64
******************************
False    2451
True      129
Name: GarageYrBlt, dtype: int64
******************************
False    2579
True        1
Name: GarageCars, dtype: int64
******************************
False    2579
True        1
Name: GarageArea, dtype: int64
******************************
False    2580
Name: WoodDeckSF, dtype: int64
******************************
False    2580
Name: OpenPorchSF, dtype: int64
******************************
False    2580
Name: EnclosedPorch, dtype: int64
******************************
False    2580
Name: X3SsnPorch, dtype: int64
******************************
False    2580
Name: ScreenPorch, dtype: int64
******************************
False    2580
Name: PoolArea, dtype: int64
******************************
False    2580
Name: MiscVal, dtype: in

In [13]:
housing3['Fireplaces'].value_counts()

0    1241
1    1132
2     195
3      11
4       1
Name: Fireplaces, dtype: int64

In [19]:
housing3['FireplaceQu'] = housing3['FireplaceQu'].fillna('None')

In [84]:
# Impute None to Nan value in columns
impute_feature = housing3[['FireplaceQu','GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond']]
for feature in impute_feature.columns:
    housing3[feature] = housing3[feature].fillna('None')

In [33]:
# housing3['GarageType'].value_counts()

Attchd     1527
Detchd      716
BuiltIn     153
None        127
Basment      27
2Types       21
CarPort       9
Name: GarageType, dtype: int64

In [85]:
housing3[housing3['GarageType'].isnull() == True].T

""
KitchenAbvGr
KitchenQual
TotRmsAbvGrd
Functional
Fireplaces
FireplaceQu
GarageType
GarageYrBlt
GarageFinish
GarageCars


In [88]:
# Drop the Nan from GarageCars column (1 observation)
housing3.drop(housing3[housing3['GarageCars'].isnull() == True].index,inplace=True)
housing3.columns

Index(['KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional',
       'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscFeature', 'MiscVal',
       'MoSold', 'YrSold', 'SaleType', 'SaleCondition', 'SalePrice'],
      dtype='object')

In [87]:
housing3.isnull().sum()

KitchenAbvGr        0
KitchenQual         0
TotRmsAbvGrd        0
Functional          0
Fireplaces          0
FireplaceQu         0
GarageType          0
GarageYrBlt         0
GarageFinish        0
GarageCars          0
GarageArea          0
GarageQual          0
GarageCond          0
PavedDrive          0
WoodDeckSF          0
OpenPorchSF         0
EnclosedPorch       0
3SsnPorch           0
ScreenPorch         0
PoolArea            0
PoolQC           2568
Fence            2053
MiscFeature      2480
MiscVal             0
MoSold              0
YrSold              0
SaleType            0
SaleCondition       0
SalePrice           0
dtype: int64

In [96]:
X.isnull().sum().sum()

0

#### PoolArea, PoolQC, Fence, MiscFeature, MiscVal will be drop due to too many missing value

In [89]:
housing_features2 = housing3.drop(['PoolArea','PoolQC','Fence','MiscFeature','MiscVal','SalePrice'],axis=1)
len(housing_features2.columns)

23

In [99]:
X = pd.get_dummies(housing_features2, drop_first=False, dummy_na=True)
price = housing3['SalePrice']
price.shape

(2577,)

In [80]:
X.reset_index()

,index,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,...,SaleType_VWD,SaleType_WD,SaleType_nan,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_nan
0,1,1,4,1,1939.0,2.0,399.0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,2,1,5,0,1984.0,1.0,266.0,0,105,0,...,0,1,0,0,0,0,0,1,0,0
2,3,1,5,0,1930.0,1.0,216.0,154,0,42,...,0,1,0,0,0,0,0,1,0,0
3,4,1,6,0,1940.0,1.0,281.0,0,0,168,...,0,1,0,0,0,0,0,1,0,0
4,5,1,6,0,2001.0,2.0,528.0,0,45,0,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2575,759,1,4,1,1916.0,1.0,192.0,0,98,0,...,0,1,0,0,0,0,0,1,0,0
2576,760,1,8,1,1955.0,2.0,452.0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2577,761,2,8,0,1949.0,3.0,871.0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2578,762,1,7,1,2000.0,2.0,486.0,193,96,0,...,0,1,0,0,0,0,0,1,0,0


In [103]:
lin_reg = LinearRegression().fit(X,price)
lin_reg.score(X,price)

0.7938900509000838

In [114]:
tree_reg = DecisionTreeRegressor().fit(X,price)
tree_reg.score(X,price)

0.9999886178627502

In [115]:
forest_reg = RandomForestRegressor().fit(X,price)
forest_reg.score(X,price)

0.9695221483370355